In [24]:
# the subbort that had a major move will be considerred more reliable
# same for resistance

In [25]:
import pandas as pd
import numpy as np
# import yfinance

# from mplfinance import candlestick_ohlc
from mplfinance.original_flavor import candlestick_ohlc

import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
import os
import plotly.graph_objects as go

from datetime import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [26]:
# plt.rcParams['figure.figsize'] = [12, 7]

# plt.rc('font', size=14) 

In [ ]:

name = 'BTC-USD'
df=pd.read_csv("C:\\Users\Grant\Desktop\work_git\work\data\\4h\ZILUSDT_4h.csv",index_col=['Time'], parse_dates=['Time'])


# ticker = yfinance.Ticker(name)
# df = ticker.history(interval="90m")

# # df = ticker.history(interval="1d",start="2020-03-15",end="2020-07-15")



# df = df.reset_index().set_index('Time') # SWITCH INDEX AND TIME COLumn places

# df = df.set_index('Time')

# df['Date'] = df['Time']
# pd.to_datetime(df['Time'])
# pd.to_datetime(df['Date'])
# df = df.set_index('Time')


In [ ]:

# df

In [ ]:
df['Date'] = pd.to_datetime(df.index)
# change datetime into matplotlib time
# does not work with plotly
# df['Date'] = df['Date'].apply(mpl_dates.date2num)

df = df.loc[:,['Date' ,'Open', 'High', 'Low', 'Close']]

In [ ]:
# df


In [ ]:
def isSupport(df,i):
  # fractal (i-2)>(i-1)>i<(i+1)<(i+2)
  support = df['Low'][i] < df['Low'][i-1]  and df['Low'][i] < df['Low'][i+1] \
  and df['Low'][i+1] < df['Low'][i+2] and df['Low'][i-1] < df['Low'][i-2]

  return support

def isResistance(df,i):
  resistance = df['High'][i] > df['High'][i-1]  and df['High'][i] > df['High'][i+1] \
  and df['High'][i+1] > df['High'][i+2] and df['High'][i-1] > df['High'][i-2] 

  return resistance

In [ ]:
levels = []
# start from 2 because i need i-2
# end at shape[0]-2 because  i need i+2
# i is the number of current row
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    levels.append((i,df['Low'][i]))
  elif isResistance(df,i):
    levels.append((i,df['High'][i]))

In [ ]:
# MOCHLV = zip(df.Date, df.Open, df.Close, df.High, df.Low, df.Volume)


In [ ]:
# def plot_all():
#   fig, ax = plt.subplots()

#   candlestick_ohlc(ax,df.values,width=0.6, \
#                    colorup='green', colordown='red', alpha=0.8)
#   # candlestick_ohlc(ax,MOCHLV)

#   date_format = mpl_dates.DateFormatter('%d %b %Y')
#   ax.xaxis.set_major_formatter(date_format)
#   fig.autofmt_xdate()

#   fig.tight_layout()

#   for level in levels:
#     plt.hlines(level[1],xmin=df['Date'][level[0]],\
#                xmax=max(df['Date']),colors='blue')
#   fig.show()

In [ ]:
def plot_all():

    fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    for level in levels:
        # fig.add_hline(level[1])
        fig.add_shape(type='line',
                x0=df['Date'][level[0]],
                y0=level[1],
                x1=max(df['Date']),
                y1=level[1],
                line=dict(color='black',),
                xref='x',
                yref='y')
    
    fig.layout.xaxis.rangeslider.visible = False


    fig.show()



#   fig, ax = plt.subplots()

#   candlestick_ohlc(ax,df.values,width=0.6, \
#                    colorup='green', colordown='red', alpha=0.8)
#   # candlestick_ohlc(ax,MOCHLV)

#   date_format = mpl_dates.DateFormatter('%d %b %Y')
#   ax.xaxis.set_major_formatter(date_format)
#   fig.autofmt_xdate()

#   fig.tight_layout()

#   for level in levels:
#     plt.hlines(level[1],xmin=df['Date'][level[0]],\
#                xmax=max(df['Date']),colors='blue')
#   fig.show()



In [ ]:
# plot_all()

In [ ]:
s =  np.mean(df['High'] - df['Low'])

In [ ]:
def isFarFromLevel(l):
  return np.sum([abs(l-x) < s  for x in levels]) == 0

In [ ]:
levels = []
for i in range(2,df.shape[0]-2):
  if isSupport(df,i):
    l = df['Low'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

  elif isResistance(df,i):
    l = df['High'][i]

    if isFarFromLevel(l):
      levels.append((i,l))

In [ ]:
levels

In [ ]:
# plot_all()

In [ ]:
for level in levels:
    if (df['Close'][-2] < level[1]) and (df['Close'][-1] > level[1]):
        print(level[1] + 'breakout')
